**Enabling a Free GPU/TPU**

It is very simple to alter default hardware (CPU to GPU/TPU or vice versa):

Click Edit > Notebook settings > Change runtime type and select GPU or TPU as Hardware accelerator.




In [0]:
# Install PyTorch
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [43]:
# Is GPU working? Double check to see if it's working:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
# Import a local file (fc_model.py) to Google Colab. 
# https://stackoverflow.com/questions/49417985/import-local-file-to-google-colab?rq=1
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import torch
from torchvision import datasets, transforms
import torchvision.models as models
import helper
from torch import nn, optim
import torch.nn.functional as F
import fc_model

In [45]:
# It's important to downgrade Pillow otherwise we'll get an AttributeError 
# https://stackoverflow.com/questions/48547660/attributeerror-module-pil-image-has-no-attribute-register-extensions
!pip install Pillow==4.0.0

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.4.1
    Uninstalling Pillow-5.4.1:
      Successfully uninstalled Pillow-5.4.1


In [0]:
# The question is how to upload large files to Google Colab and remote Jupyter notebooks?
# Sending files through google drive seems to be a hassle so I just created a Github repository 
# with the data can create a GitHub repository with the data I want to transfer. GitHub has a 
# hard limit of 25MB per file and a soft limit of 1GB per repository so with cats and dogs 
# that's fine. Once you create the repository, you can just clone it in Google Colab
# more tutorials: https://medium.freecodecamp.org/how-to-transfer-large-files-to-google-colab-and-remote-jupyter-notebooks-26ca252892fa

!git clone https://github.com/adelekuzmiakova/cats_and_dogs_filtered.git

Cloning into 'cats_and_dogs_filtered'...
remote: Enumerating objects: 3013, done.
remote: Counting objects: 100% (3013/3013), done.
remote: Compressing objects: 100% (3011/3011), done.
remote: Total 3013 (delta 0), reused 3010 (delta 0), pack-reused 0
Receiving objects: 100% (3013/3013), 64.93 MiB | 47.12 MiB/s, done.


In [46]:
ls -a

./   cats_and_dogs_filtered/  fc_model.py          __pycache__/
../  .config/                 .ipynb_checkpoints/  sample_data/


In [0]:
base_dir = 'cats_and_dogs_filtered/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [0]:
input_size = [224, 224]
channel_mean = [0.485, 0.456, 0.406]
channel_std = [0.229, 0.224, 0.225]
train_transforms = transforms.Compose([transforms.RandomResizedCrop(input_size[0]),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(channel_mean, channel_std)]) 
# no data augmentation on the test data:
test_transforms = transforms.Compose([transforms.RandomResizedCrop(input_size[0]),
                                      transforms.ToTensor(),
                                      transforms.Normalize(channel_mean, channel_std)]) 
# pass the transforms:
train_data = datasets.ImageFolder(train_dir, transform = train_transforms)
test_data = datasets.ImageFolder(validation_dir, transform = test_transforms)

In [0]:
batch_size = 32
trainloader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size = batch_size)

In [50]:
model = models.densenet121(pretrained = True)

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [51]:
for param in model.parameters():
    param.requires_grad = False
    
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 256)),
    ('relu', nn.ReLU()),
    ('dropout1', nn.Dropout(.25)),
    ('fc2', nn.Linear(256, 2)),
    ('output', nn.LogSoftmax(dim = 1))    
]))

model.classifier = classifier
criterion = nn.NLLLoss()
# we train only the classifier since feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr = 0.001)
model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [52]:
epochs = 10
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
  
    # Training loop:
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # Validation loop after 5 training batches:
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print('Epoch # : ' + str(epoch) + ' of ' +str(epochs-1) + ', Train loss: ' + str(running_loss/print_every) +
                 ', Test loss: ' + str(test_loss/len(testloader)) + ', Test accuracy: ' + str(accuracy/len(testloader)))
            running_loss = 0
            model.train()

Epoch # : 0 of 9, Train loss: 0.5675344705581665, Test loss: 0.329380392562598, Test accuracy: 0.9423828125
Epoch # : 0 of 9, Train loss: 0.30878494381904603, Test loss: 0.29509246558882296, Test accuracy: 0.8779296875
Epoch # : 0 of 9, Train loss: 0.2026769697666168, Test loss: 0.28230123734101653, Test accuracy: 0.857421875
Epoch # : 0 of 9, Train loss: 0.27689010798931124, Test loss: 0.17554302426287904, Test accuracy: 0.9140625
Epoch # : 0 of 9, Train loss: 0.34564300179481505, Test loss: 0.14792603778187186, Test accuracy: 0.9365234375
Epoch # : 0 of 9, Train loss: 0.15045301765203475, Test loss: 0.1319840003270656, Test accuracy: 0.9501953125
Epoch # : 0 of 9, Train loss: 0.24986507892608642, Test loss: 0.1360641893115826, Test accuracy: 0.9462890625
Epoch # : 0 of 9, Train loss: 0.12512931004166603, Test loss: 0.13585329573834315, Test accuracy: 0.9404296875
Epoch # : 0 of 9, Train loss: 0.0914824739098549, Test loss: 0.15159329981543124, Test accuracy: 0.9306640625
Epoch # : 0 

KeyboardInterrupt: ignored

1